In [ ]:
import calendar
from copy import deepcopy
import csv
import datetime
import os

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz

import matplotlib
import matplotlib.pyplot as plt

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

from pylab import rcParams

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import stats

import seaborn as sns

import pytz

!pip install influxdb
import influxdb

HOST = 'influx.linklab.virginia.edu'
PORT = 443
USERNAME = 'cps1f23'
PASSWORD = 'phah7goohohng5ooL9mae1quohpei1Ahsh1uGing'
DATABASE = 'gateway-generic'

client = influxdb.InfluxDBClient(HOST, PORT, USERNAME, PASSWORD, DATABASE, ssl=True, verify_ssl=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
eData = client.query("SELECT MEAN(value) FROM \"power_w\" WHERE time >= '2024-08-01T00:00:00Z' AND time <= '2024-11-01T00:00:00Z' AND location_specific = '281 Olsson' GROUP BY time(10m),device_id,location_specific")
cData = client.query("SELECT MEAN(value) FROM \"co2_ppm\" WHERE time >= '2024-08-01T00:00:00Z' AND time <= '2024-11-01T00:00:00Z' AND location_specific = '281 Olsson' GROUP BY time(10m),device_id,location_specific")

eDF = pd.DataFrame(columns=['time','power','device_id','location_specific'])
cDF = pd.DataFrame(columns=['time','co2','device_id','location_specific'])

n = 0
for i in eData:
  for j in i:
    newLine = pd.DataFrame({'time':[j['time']],
                            'power':[j['mean']],
                            'device_id':[eData.keys()[n][1]['device_id']],
                            'location_specific':[eData.keys()[n][1]['location_specific']]})
    eDF = pd.concat([eDF, newLine],ignore_index=True)
  n += 1

n = 0
for i in cData:
  for j in i:
    newLine = pd.DataFrame({'time':[j['time']],
                            'co2':[j['mean']],
                            'device_id':[cData.keys()[n][1]['device_id']],
                            'location_specific':[cData.keys()[n][1]['location_specific']]})
    cDF = pd.concat([cDF, newLine],ignore_index=True)
  n += 1

eDF['power'] = eDF['power'].ffill()
cDF['co2'] = cDF['co2'].ffill()

fullDF = pd.merge(eDF, cDF, on=['time','location_specific'])
eDF

<ipython-input-2-2728bfd8b179>:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eDF = pd.concat([eDF, newLine],ignore_index=True)
<ipython-input-2-2728bfd8b179>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cDF = pd.concat([cDF, newLine],ignore_index=True)


,time,power,device_id,location_specific
0,2024-08-01T00:00:00Z,61.725996,ELEUV0202SM1Ja,245 Olsson
1,2024-08-01T00:10:00Z,21.619996,ELEUV0202SM1Ja,245 Olsson
2,2024-08-01T00:20:00Z,10.978879,ELEUV0202SM1Ja,245 Olsson
3,2024-08-01T00:30:00Z,10.069404,ELEUV0202SM1Ja,245 Olsson
4,2024-08-01T00:40:00Z,60.750675,ELEUV0202SM1Ja,245 Olsson
...,...,...,...,...
13244,2024-10-31T23:20:00Z,97.263191,ELEUV0202SM1Ja,245 Olsson
13245,2024-10-31T23:30:00Z,62.911415,ELEUV0202SM1Ja,245 Olsson
13246,2024-10-31T23:40:00Z,16.249832,ELEUV0202SM1Ja,245 Olsson
13247,2024-10-31T23:50:00Z,11.271587,ELEUV0202SM1Ja,245 Olsson


In [ ]:
fullDF.to_csv('data.csv')

In [ ]:
eData = client.query("SELECT MEAN(value) FROM \"power_w\" WHERE time >= '2024-08-01T00:00:00Z' AND time <= '2024-11-01T00:00:00Z' AND location_specific = '243 Olsson' GROUP BY time(10m),device_id,location_specific")
cData = client.query("SELECT MEAN(value) FROM \"co2_ppm\" WHERE time >= '2024-08-01T00:00:00Z' AND time <= '2024-11-01T00:00:00Z' AND location_specific = '243 Olsson' GROUP BY time(10m),device_id,location_specific")

eDF = pd.DataFrame(columns=['time','power','device_id','location_specific'])
cDF = pd.DataFrame(columns=['time','co2','device_id','location_specific'])

n = 0
for i in eData:
  for j in i:
    newLine = pd.DataFrame({'time':[j['time']],
                            'power':[j['mean']],
                            'device_id':[eData.keys()[n][1]['device_id']],
                            'location_specific':[eData.keys()[n][1]['location_specific']]})
    eDF = pd.concat([eDF, newLine],ignore_index=True)
  n += 1

n = 0
for i in cData:
  for j in i:
    newLine = pd.DataFrame({'time':[j['time']],
                            'co2':[j['mean']],
                            'device_id':[cData.keys()[n][1]['device_id']],
                            'location_specific':[cData.keys()[n][1]['location_specific']]})
    cDF = pd.concat([cDF, newLine],ignore_index=True)
  n += 1

eDF['power'] = eDF['power'].ffill()
cDF['co2'] = cDF['co2'].ffill()

fullDF = pd.merge(eDF, cDF, on=['time','location_specific'])
fullDF.to_csv('243_data_1.csv')

<ipython-input-40-dca28fbf0e27>:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eDF = pd.concat([eDF, newLine],ignore_index=True)
<ipython-input-40-dca28fbf0e27>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cDF = pd.concat([cDF, newLine],ignore_index=True)


In [ ]:
fullDF

,time,power,device_id_x,location_specific,co2,device_id_y
0,2024-08-01T00:00:00Z,2.548154,ELEUV0202SM2Ja,243 Olsson,447.783333,70886b123bc8
1,2024-08-01T00:10:00Z,2.150196,ELEUV0202SM2Ja,243 Olsson,444.283333,70886b123bc8
2,2024-08-01T00:20:00Z,2.386381,ELEUV0202SM2Ja,243 Olsson,445.152542,70886b123bc8
3,2024-08-01T00:30:00Z,2.373820,ELEUV0202SM2Ja,243 Olsson,444.300000,70886b123bc8
4,2024-08-01T00:40:00Z,2.419661,ELEUV0202SM2Ja,243 Olsson,449.857143,70886b123bc8
...,...,...,...,...,...,...
13244,2024-10-31T23:20:00Z,2.504077,ELEUV0202SM2Ja,243 Olsson,437.288136,70886b123bc8
13245,2024-10-31T23:30:00Z,2.226016,ELEUV0202SM2Ja,243 Olsson,435.379310,70886b123bc8
13246,2024-10-31T23:40:00Z,2.444588,ELEUV0202SM2Ja,243 Olsson,436.649123,70886b123bc8
13247,2024-10-31T23:50:00Z,2.261514,ELEUV0202SM2Ja,243 Olsson,438.466667,70886b123bc8


In [ ]:
eData = client.query("SELECT MEAN(value) FROM \"PIR Status\" WHERE time > now() - 1d AND location_specific = '241 Olsson' GROUP BY time(10m),device_id,location_specific")
cData = client.query("SELECT MEAN(value) FROM \"co2_ppm\" WHERE time > now() - 1d AND location_specific = '241 Olsson' GROUP BY time(10m),device_id,location_specific")

eDF = pd.DataFrame(columns=['time','power','device_id','location_specific'])
cDF = pd.DataFrame(columns=['time','co2','device_id','location_specific'])

n = 0
for i in eData:
  for j in i:
    newLine = pd.DataFrame({'time':[j['time']],
                            'power':[j['mean']],
                            'device_id':[eData.keys()[n][1]['device_id']],
                            'location_specific':[eData.keys()[n][1]['location_specific']]})
    eDF = pd.concat([eDF, newLine],ignore_index=True)
  n += 1

n = 0
for i in cData:
  for j in i:
    newLine = pd.DataFrame({'time':[j['time']],
                            'co2':[j['mean']],
                            'device_id':[cData.keys()[n][1]['device_id']],
                            'location_specific':[cData.keys()[n][1]['location_specific']]})
    cDF = pd.concat([cDF, newLine],ignore_index=True)
  n += 1

eDF['power'] = eDF['power'].ffill()
cDF['co2'] = cDF['co2'].ffill()

fullDF = pd.merge(eDF, cDF, on=['time','location_specific'])
fullDF.to_csv('269_data_1.csv')

<ipython-input-4-2c5e7644fa74>:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eDF = pd.concat([eDF, newLine],ignore_index=True)
<ipython-input-4-2c5e7644fa74>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cDF = pd.concat([cDF, newLine],ignore_index=True)


In [ ]:
fullDF.to_csv('281_data_2021_1.csv')